In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
#import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [2]:
### Load .CSV and 
### subset bleaching events (conditional, e.g. exclude NaN)
### Filter only CodeID, site names and country" 
df = pd.read_csv('CBdbV2.csv',encoding='latin-1', low_memory=False)
df = df[['RECORD_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','DATE','MONTH','YEAR']]
df = pd.DataFrame(df)
#### Convert index into column
#df.reset_index(level=0, inplace=True)
df['ITEM_ID'] = df.index.astype(str).str.zfill(7)   # add '0s' at the beginning of the number and convert as string 
df = df[pd.to_numeric(df['MONTH'], errors='coerce').notnull()] # Drop rows that do not have as.numeric DATE (DAY)
df['MONTH'] = df['MONTH'].apply(lambda x: int(float(x)))
df['MONTH'] = df['MONTH'].apply('{0:0>2}'.format)
df['DUMMY_DAY'] = df['MONTH'].apply(lambda x: 15 if x == '02' else 16) 
df['dummy_date'] = df.apply(lambda x: '%s-%s-%s' % (x['YEAR'], int(float(x['MONTH'])), x['DUMMY_DAY']), axis=1) # create a column in datetime format; %s - String (or any object with a string representation, like numbers), %d - Integers %f - Floating point numbers
df['dummy_date'] = df['dummy_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) # use datetime funtion to convert string 'full_date' to datetime 
### Re-order by column starting by 'ID' and renaming LAT and LON
df['dummy_date'] = pd.to_datetime(df['dummy_date']).dt.to_period('M')
df['dummy_date'] = df['dummy_date'].apply(lambda x: x.to_timestamp()) # convert Period M to timestamp 
df = df.reindex(columns=['ITEM_ID','COUNTRY','LAT_corrected','LON_corrected','SEVERITY_CODE','MONTH','YEAR','dummy_date'])
df.rename(columns={'LAT_corrected': 'lat', 'LON_corrected': 'lon'}, inplace=True) # use "implace = True" to rename the existing DataFrame (rather than creating a copy)
df = df.dropna(axis=0, how='any')  # Drop row with NaN if any

In [3]:
df = df.loc[(df['YEAR'] >= 2000) & (df['YEAR'] <= 2017)] 
len(df) # the value should be 35,678 (# rows) 

34618

In [4]:
df['dummy_date']

3053    2001-02-01
3054    2001-02-01
3055    2001-02-01
3056    2001-02-01
3057    2001-02-01
           ...    
37769   2017-12-01
37770   2017-12-01
37771   2017-12-01
37772   2017-12-01
37773   2017-12-01
Name: dummy_date, Length: 34618, dtype: datetime64[ns]

In [5]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) # open_dataset works fine with either full path or filename.nc 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,time=time,method='nearest') 
    rsds = dsloc['sfc_sw_down_all_mon']
    return rsds 

In [6]:
home = Path(expanduser("~"))
x = []
for row in df.itertuples():
        lat = row.lat,
        lon = row.lon
        time = row.dummy_date
        path = home / Path("F:/Chapter_3/RSDS/CERES/CERES_EBAF-sfc_sw_down_anom.nc")
        array = file_info(path)
        rsds_val = array_sel(array)
        x.append(rsds_val)
rsds = [item[0].values for item in x]
df['RSDS'] = rsds
df.to_csv('RSDS.csv')

Timestamp('2017-12-01 00:00:00')